In [ ]:
import spacy
import pandas as pd

import unidecode 


# from spacy.lang.nl import STOP_WORDS
# from string import punctuation
# from collections import Counter
# from heapq import nlargest


In [ ]:
import transformers
from transformers import pipeline


from transformers import RobertaTokenizer, RobertaForSequenceClassification
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robbert-v2-dutch-base")
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robbert-v2-dutch-base")





In [ ]:
sequence='brussels hoofdstedelijk gewest januari . ministerieel besluit tot vaststelling van de modellen van formulier bedoeld in artikel , ss van het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen de minister van de brusselse hoofdstedelijke regering, belast met financien en begroting, gelet op het wetboek der successierechten, artikel bis , ss , ingevoegd door de ordonnantie van december houdende het tweede deel van de fiscale hervorming gelet op het besluit van de brusselse hoofdstedelijke regering van januari tot vaststelling van de modaliteiten van het gunstregime toepasselijk op de successierechten bij overdracht van familiale ondernemingen en familiale vennootschappen, artikel gelet op de gelijkekansentest uitgevoerd in toepassing van artikel van de ordonnantie van oktober tot invoering van de gelijkekansentest overwegende dat artikel van bovengenoemd besluit stelt dat de minister van de brusselse hoofdstedelijke regering, belast met financien en begroting, bevoegd is om het formulier dat bedoeld is om na te gaan of de voorwaarden voor het behoud van de het fiscale gunstregime, dat werd bekomen in het kader van de overdracht van een familiale onderneming of een familiale vennootschap, vervuld zijn gebleven tot het einde van de driejarige periode die aanving bij het overlijden van de decujus, zoals bedoeld in artikel bis , ss , van het wetboek der successierechten overwegende dat dit ministerieel besluit geen reglementaire voorschriften bevat in de zin van artikel , ss , eerste lid, van de gecoordineerde wetten van de raad van state van januari , is het niet onderworpen aan het advies van de raad van state, afdeling wetgeving, besluit : artikel . het formulier dat bedoeld is om na te gaan of de voorwaarden voor het behoud van de het fiscale gunstregime, dat werd bekomen in het kader van de overdracht van een familiale onderneming, vervuld zijn gebleven tot het einde van de driejarige periode die aanving bij het overlijden van de decujus, zoals bedoeld in artikel bis , ss van het wetboek der successierechten, is hernomen in bijlage van dit besluit. art. . het formulier dat bedoeld is om na te gaan of de voorwaarden voor het behoud van het fiscale gunstregime, dat werd bekomen in het kader van het overdracht van een familiale vennootschap, vervuld zijn gebleven tot het einde van de driejarige periode die aanving bij het overlijden van de decujus, zoals bedoeld in artikel bis , ss van het wetboek der successierechten, is hernomen in bijlage bij dit besluit. art. . dit besluit heeft uitwerking met ingang van januari . brussel, januari . de minister van de brusselse hoofdstedelijke regering, belast met financien, begroting, openbaar ambt en de promotie van meertaligheid, s. gatz voor de raadpleging van de tabel, zie beeld'




In [ ]:
labels=['Aanslagjaar','Aftrekbarebesteding','afvalbelastingen','Bedrijfsvoorheffing','belastbaartijdperk','belastingaangifte','belastingenopberoepsinkomsten','belastingenoponroerendegoederen','belastingkorting','Belastingkrediet','Belastingontwijking','belastingoverdetoegevoegdewaarde','belastingteruggave','belastingvermindering','Belastingvrijesom','Belcotax-on-web','Biztax','Bronbelasting','BTW','Decumul','FinProf','fiscaalvoordeel','fiscalebepalingenbelastingontduiking','fiscalehervorming','fiscaleschuldvorderingen','fiscaleschuldvorderingen','fiscalestimulans','Forfaitaireberoepskosten','Inkomstenbelastingen','Intervat','kadastraalinkomen','milieutaksen','onroerendevoorheffing','Opcentiemen','Progressievebelastingtarieven','provinciebelastingen','rechtspersonenbelasting','roerendevoorheffing','successierechten','vennootschapsbelasting','verbruiksbelastingen','verkeersbelastingen'
]

In [ ]:
classifier = pipeline("zero-shot-classification", 
                      model=model,tokenizer=tokenizer)

#sequence = "L'équipe de France joue aujourd'hui au Parc des Princes"
#candidate_labels = ["sport","politique","science"]
hypothesis_template = "Ce texte parle de {}."    

classifier(sequence, labels, hypothesis_template=hypothesis_template)     
                                    

In [ ]:
df=pd.read_csv("data_03.csv",sep="|")

In [ ]:
l=df['nl_body']

In [ ]:
def reducing_incorrect_character_repeatation(text):
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

In [ ]:
import re
import string

documents_clean = []
for d in l[0:6]:
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', d)
    


    document_test= unidecode.unidecode(d)
    document_test = document_test.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    

       
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', document_test)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    document_test = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    document_test = document_test.lower()
    document_test=reducing_incorrect_character_repeatation(document_test)

    document_test = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', document_test) 

   
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    documents_clean.append(document_test)

In [ ]:
documents_clean[0]

In [ ]:
text=documents_clean[0]
text

In [ ]:
from transformers import pipeline

# Init translator
translator = pipeline("translation_nl_to_en",model=model,tokenizer=tokenizer)

# Translate text
#text = "Hello my friends! How are you doing today?"
translation = translator(text)

# Print translation
print(translation)
